In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/Miyu/opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.9.0                |   py38h5347e94_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    python_abi-3.8             |           1_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geograp

In [3]:
CLIENT_ID = 'IEO3BCOMPMVDENSQXFGQOO21FUGTKIV1ZX01F4LNKLV0PK25' # your Foursquare ID
CLIENT_SECRET = 'XNJPG4KKQYJR2ZJM1PFS1FBAOM3APZ32TT2KDDQ3QNBPWCV0' # your Foursquare Secret
VERSION = '20200628'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IEO3BCOMPMVDENSQXFGQOO21FUGTKIV1ZX01F4LNKLV0PK25
CLIENT_SECRET:XNJPG4KKQYJR2ZJM1PFS1FBAOM3APZ32TT2KDDQ3QNBPWCV0


In [87]:
district=["Batu Ferringhi",
"Tanjung Bungah",
"Tanjung Tokong",
"Pulau Tikus",
"Batu Lanchang",
"Air Itam",
"Paya Terubong",
"Jelutong",
"Gelugor",
"Georgetown",
"Bayan Baru"]

In [88]:
df_district=pd.DataFrame(district,columns=['District'])
j=0

for i in district:
    address = i+","+"Penang"
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude= location.longitude
    df_district.loc[j,'Latitude']=latitude
    df_district.loc[j,'Longitude']=longitude
    j=j+1
    print(latitude, longitude)
    
df_district

5.4782177 100.2687609
5.4621634 100.286995
5.4461393 100.3052541
5.4318216 100.3117683
5.3903225 100.3061094
5.388131400000001 100.27869084474452
5.3718034 100.276162
5.3873462 100.3145585
5.3675141 100.3121321
5.4145681 100.3298035
5.3248811 100.285231


,District,Latitude,Longitude
0,Batu Ferringhi,5.478218,100.268761
1,Tanjung Bungah,5.462163,100.286995
2,Tanjung Tokong,5.446139,100.305254
3,Pulau Tikus,5.431822,100.311768
4,Batu Lanchang,5.390322,100.306109
5,Air Itam,5.388131,100.278691
6,Paya Terubong,5.371803,100.276162
7,Jelutong,5.387346,100.314559
8,Gelugor,5.367514,100.312132
9,Georgetown,5.414568,100.329803


In [113]:
import folium

# create map of Toronto using latitude and longitude values
map_pg = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, district in zip(df_district['Latitude'], df_district['Longitude'],df_district['District'] ):
    label = '{}'.format(district)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        ).add_to(map_pg)  
    
map_pg

In [114]:
neighborhood_latitude = df_district.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_district.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_district.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Batu Ferringhi are 5.4782177, 100.2687609.


In [115]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IEO3BCOMPMVDENSQXFGQOO21FUGTKIV1ZX01F4LNKLV0PK25&client_secret=XNJPG4KKQYJR2ZJM1PFS1FBAOM3APZ32TT2KDDQ3QNBPWCV0&v=20200628&ll=5.4782177,100.2687609&radius=500&limit=100'

In [116]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f96b5c1c0877943bcfd8a3c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 12,
  'suggestedBounds': {'ne': {'lat': 5.482717704500004,
    'lng': 100.27327311171317},
   'sw': {'lat': 5.473717695499996, 'lng': 100.26424868828684}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d1ac5522a52b60c6e5df2ff',
       'name': 'Miami Beach',
       'location': {'address': 'Jalan Batu Ferringhi',
        'lat': 5.477834004656061,
        'lng': 100.26765957145422,
        'labeledLatLngs': [{'label': 'display',
          'lat': 5.47

In [117]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [118]:
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-118-4cac159fdf56>:6: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Miami Beach,Beach,5.477834,100.267660
1,Island Resort,Residential Building (Apartment / Condo),5.477299,100.265781
2,Teddyville Museum,Recreation Center,5.477220,100.269008
3,DoubleTree by Hilton,Hotel,5.477699,100.269027
4,Axis Lounge,Hotel Bar,5.477629,100.269290


In [119]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [120]:
pg_venues = getNearbyVenues(names=df_district['District'],
                                   latitudes=df_district['Latitude'],
                                   longitudes=df_district['Longitude']
                                  )

Batu Ferringhi
Tanjung Bungah
Tanjung Tokong
Pulau Tikus
Batu Lanchang
Air Itam
Paya Terubong
Jelutong
Gelugor
Georgetown
Bayan Baru


In [121]:
print(pg_venues.shape)
pg_venues

(400, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Batu Ferringhi,5.478218,100.268761,Miami Beach,5.477834,100.267660,Beach
1,Batu Ferringhi,5.478218,100.268761,Island Resort,5.477299,100.265781,Residential Building (Apartment / Condo)
2,Batu Ferringhi,5.478218,100.268761,Teddyville Museum,5.477220,100.269008,Recreation Center
3,Batu Ferringhi,5.478218,100.268761,DoubleTree by Hilton,5.477699,100.269027,Hotel
4,Batu Ferringhi,5.478218,100.268761,Axis Lounge,5.477629,100.269290,Hotel Bar
...,...,...,...,...,...,...,...
395,Bayan Baru,5.324881,100.285231,家乐福中餐厅 D Colorful Chinese Restaurant,5.323315,100.281769,Chinese Restaurant
396,Bayan Baru,5.324881,100.285231,Sushi Edo,5.322838,100.281794,Japanese Restaurant
397,Bayan Baru,5.324881,100.285231,Ruyi Delights,5.328415,100.284249,Chinese Restaurant
398,Bayan Baru,5.324881,100.285231,Elit Height open lounge,5.322781,100.282315,Park


In [122]:
pg_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Air Itam,26,26,26,26,26,26
Batu Ferringhi,12,12,12,12,12,12
Batu Lanchang,34,34,34,34,34,34
Bayan Baru,58,58,58,58,58,58
Gelugor,10,10,10,10,10,10
Georgetown,61,61,61,61,61,61
Jelutong,32,32,32,32,32,32
Paya Terubong,6,6,6,6,6,6
Pulau Tikus,100,100,100,100,100,100


In [123]:
# one hot encoding
pg_onehot = pd.get_dummies(pg_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pg_onehot['Neighbourhood'] = pg_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [pg_onehot.columns[-1]] + list(pg_onehot.columns[:-1])
pg_onehot = pg_onehot[fixed_columns]

pg_onehot.head()

,Neighbourhood,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Beach,Bed & Breakfast,Beer Bar,Bike Shop,...,Street Food Gathering,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Batu Ferringhi,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Batu Ferringhi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Batu Ferringhi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Batu Ferringhi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Batu Ferringhi,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [124]:
pg_onehot.shape

(400, 125)

In [125]:
pg_grouped = pg_onehot.groupby('Neighbourhood').mean().reset_index()
pg_grouped.head()

,Neighbourhood,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Bakery,Beach,Bed & Breakfast,Beer Bar,Bike Shop,...,Street Food Gathering,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wine Bar,Women's Store
0,Air Itam,0.0,0.0,0.000000,0.038462,0.076923,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.038462,0.0,0.0
1,Batu Ferringhi,0.0,0.0,0.000000,0.000000,0.000000,0.166667,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,Batu Lanchang,0.0,0.0,0.000000,0.029412,0.088235,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.088235,0.0,0.000000,0.0,0.0
3,Bayan Baru,0.0,0.0,0.017241,0.017241,0.034483,0.000000,0.0,0.0,0.017241,...,0.0,0.0,0.017241,0.0,0.0,0.034483,0.0,0.017241,0.0,0.0
4,Gelugor,0.0,0.0,0.000000,0.100000,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.1,0.000000,0.0,0.000000,0.0,0.0


In [126]:
pg_grouped.shape

(11, 125)

In [127]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [128]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = pg_grouped['Neighbourhood']

for ind in np.arange(pg_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(pg_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Air Itam,Seafood Restaurant,Bakery,Coffee Shop,Cosmetics Shop,Food Court,Residential Building (Apartment / Condo),Food Stand,Pool,Sandwich Place,Burger Joint
1,Batu Ferringhi,Hotel,Beach,Gym,Residential Building (Apartment / Condo),Recreation Center,Racetrack,Pub,Boat or Ferry,Deli / Bodega,Hotel Bar
2,Batu Lanchang,Coffee Shop,Thai Restaurant,Bakery,Food Court,Fast Food Restaurant,Food Truck,Market,Malay Restaurant,Noodle House,Satay Restaurant
3,Bayan Baru,Café,Chinese Restaurant,Dessert Shop,Korean Restaurant,Noodle House,Coffee Shop,Thai Restaurant,Pharmacy,Indian Restaurant,Bakery
4,Gelugor,Food Truck,Burger Joint,Malay Restaurant,Building,National Park,Food,Tennis Court,Asian Restaurant,Hotel,Farmers Market
5,Georgetown,Dessert Shop,Vegetarian / Vegan Restaurant,Hotel,Coffee Shop,Bakery,Noodle House,Café,Chinese Restaurant,Art Museum,Food Stand
6,Jelutong,Malay Restaurant,Thai Restaurant,Coffee Shop,Bakery,Café,Hotpot Restaurant,Burger Joint,Halal Restaurant,Gym,Food Truck
7,Paya Terubong,Art Gallery,Burger Joint,Food Truck,Soup Place,Restaurant,Frozen Yogurt Shop,Fast Food Restaurant,Fish Market,Farmers Market,Food & Drink Shop
8,Pulau Tikus,Coffee Shop,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Asian Restaurant,Breakfast Spot,Boutique,Bakery,Ice Cream Shop
9,Tanjung Bungah,Chinese Restaurant,Food Truck,Café,Seafood Restaurant,Playground,Coffee Shop,Bus Stop,Convenience Store,Cosmetics Shop,Lounge


In [129]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

pg_grouped_clustering = pg_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(pg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 0, 1, 2, 1, 0, 4, 1, 1], dtype=int32)

In [130]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pg_merged = df_district

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pg_merged = pg_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='District')

pg_merged

,District,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Batu Ferringhi,5.478218,100.268761,3,Hotel,Beach,Gym,Residential Building (Apartment / Condo),Recreation Center,Racetrack,Pub,Boat or Ferry,Deli / Bodega,Hotel Bar
1,Tanjung Bungah,5.462163,100.286995,1,Chinese Restaurant,Food Truck,Café,Seafood Restaurant,Playground,Coffee Shop,Bus Stop,Convenience Store,Cosmetics Shop,Lounge
2,Tanjung Tokong,5.446139,100.305254,1,Coffee Shop,Chinese Restaurant,Café,Health & Beauty Service,Gym,Food Truck,Japanese Restaurant,Electronics Store,Grocery Store,Lounge
3,Pulau Tikus,5.431822,100.311768,1,Coffee Shop,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Asian Restaurant,Breakfast Spot,Boutique,Bakery,Ice Cream Shop
4,Batu Lanchang,5.390322,100.306109,0,Coffee Shop,Thai Restaurant,Bakery,Food Court,Fast Food Restaurant,Food Truck,Market,Malay Restaurant,Noodle House,Satay Restaurant
5,Air Itam,5.388131,100.278691,1,Seafood Restaurant,Bakery,Coffee Shop,Cosmetics Shop,Food Court,Residential Building (Apartment / Condo),Food Stand,Pool,Sandwich Place,Burger Joint
6,Paya Terubong,5.371803,100.276162,4,Art Gallery,Burger Joint,Food Truck,Soup Place,Restaurant,Frozen Yogurt Shop,Fast Food Restaurant,Fish Market,Farmers Market,Food & Drink Shop
7,Jelutong,5.387346,100.314559,0,Malay Restaurant,Thai Restaurant,Coffee Shop,Bakery,Café,Hotpot Restaurant,Burger Joint,Halal Restaurant,Gym,Food Truck
8,Gelugor,5.367514,100.312132,2,Food Truck,Burger Joint,Malay Restaurant,Building,National Park,Food,Tennis Court,Asian Restaurant,Hotel,Farmers Market
9,Georgetown,5.414568,100.329803,1,Dessert Shop,Vegetarian / Vegan Restaurant,Hotel,Coffee Shop,Bakery,Noodle House,Café,Chinese Restaurant,Art Museum,Food Stand


In [131]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

pg_merged_nonan = pg_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pg_merged_nonan['Latitude'], pg_merged_nonan['Longitude'], pg_merged_nonan['District'], pg_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [132]:
pg_merged_nonan.loc[pg_merged_nonan['Cluster Labels'] == 0, pg_merged_nonan.columns[[0] + list(range(3, pg_merged_nonan.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Batu Lanchang,0,Coffee Shop,Thai Restaurant,Bakery,Food Court,Fast Food Restaurant,Food Truck,Market,Malay Restaurant,Noodle House,Satay Restaurant
7,Jelutong,0,Malay Restaurant,Thai Restaurant,Coffee Shop,Bakery,Café,Hotpot Restaurant,Burger Joint,Halal Restaurant,Gym,Food Truck


In [133]:
pg_merged_nonan.loc[pg_merged_nonan['Cluster Labels'] == 1, pg_merged_nonan.columns[[0] + list(range(3, pg_merged_nonan.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Tanjung Bungah,1,Chinese Restaurant,Food Truck,Café,Seafood Restaurant,Playground,Coffee Shop,Bus Stop,Convenience Store,Cosmetics Shop,Lounge
2,Tanjung Tokong,1,Coffee Shop,Chinese Restaurant,Café,Health & Beauty Service,Gym,Food Truck,Japanese Restaurant,Electronics Store,Grocery Store,Lounge
3,Pulau Tikus,1,Coffee Shop,Chinese Restaurant,Café,Thai Restaurant,Noodle House,Asian Restaurant,Breakfast Spot,Boutique,Bakery,Ice Cream Shop
5,Air Itam,1,Seafood Restaurant,Bakery,Coffee Shop,Cosmetics Shop,Food Court,Residential Building (Apartment / Condo),Food Stand,Pool,Sandwich Place,Burger Joint
9,Georgetown,1,Dessert Shop,Vegetarian / Vegan Restaurant,Hotel,Coffee Shop,Bakery,Noodle House,Café,Chinese Restaurant,Art Museum,Food Stand
10,Bayan Baru,1,Café,Chinese Restaurant,Dessert Shop,Korean Restaurant,Noodle House,Coffee Shop,Thai Restaurant,Pharmacy,Indian Restaurant,Bakery


In [134]:
pg_merged_nonan.loc[pg_merged_nonan['Cluster Labels'] == 2, pg_merged_nonan.columns[[0] + list(range(3, pg_merged_nonan.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Gelugor,2,Food Truck,Burger Joint,Malay Restaurant,Building,National Park,Food,Tennis Court,Asian Restaurant,Hotel,Farmers Market


In [135]:
pg_merged_nonan.loc[pg_merged_nonan['Cluster Labels'] == 3, pg_merged_nonan.columns[[0] + list(range(3, pg_merged_nonan.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Batu Ferringhi,3,Hotel,Beach,Gym,Residential Building (Apartment / Condo),Recreation Center,Racetrack,Pub,Boat or Ferry,Deli / Bodega,Hotel Bar


In [136]:
pg_merged_nonan.loc[pg_merged_nonan['Cluster Labels'] == 4, pg_merged_nonan.columns[[0] + list(range(3, pg_merged_nonan.shape[1]))]]

,District,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Paya Terubong,4,Art Gallery,Burger Joint,Food Truck,Soup Place,Restaurant,Frozen Yogurt Shop,Fast Food Restaurant,Fish Market,Farmers Market,Food & Drink Shop
